# Müşteri Şikayet Kategorilendirme - EDA (Keşifsel Veri Analizi)

Bu notebook, küçük işletmeler için müşteri şikayet metinlerinden otomatik kategori tahmini projesinin keşifsel veri analizini içermektedir.

## Proje Hedefi
- Müşteri metinlerini şikayet doğru kategorilere atamak
- Küçük işletmelerin müşteri hizmetleri operasyonlarını otomatikleştirmek
- Müşteri memnuniyetini artırmak ve çözüm sürelerini kısaltmak

## Problem Tanımı
Küçük işletmeler, müşteri şikayetlerini manuel olarak kategorilere ayırmak zorundadır. Bu süreç:
- Zaman alıcıdır
- İnsan hatasına açıktır
- Tutarsız kategorilendirme sonuçları üretir
- Müşteri hizmetleri ekiplerinin verimliliğini düşürür

## Çözüm Yaklaşımı
NLP (Natural Language Processing) teknikleri kullanarak:
- Şikayet metinlerini analiz etmek
- Otomatik kategori tahmini yapmak
- İş süreçlerini optimize etmek

## 1. Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Kütüphaneler başarıyla yüklendi!")

## 2. Veri Setinin Yüklenmesi

In [ ]:
# Veri setini yükle
df = pd.read_csv('../data/raw/customer_complaints_full.csv')

print(f"Veri seti boyutu: {df.shape}")
print(f"Toplam kayıt sayısı: {len(df):,}")
print(f"Toplam özellik sayısı: {len(df.columns)}")

# İlk 5 satırı göster
print("\n=== İlk 5 Kayıt ===")
df.head()

## 3. Veri Seti Genel Bilgileri

In [ ]:
# Veri tiplerini incele
print("=== Veri Tipleri ===")
print(df.dtypes)

# Temel istatistikler
print("\n=== Temel İstatistikler ===")
df.describe(include='all')

In [ ]:
# Eksik değerleri kontrol et
print("=== Eksik Değerler ===")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Eksik Değer Sayısı': missing_values,
    'Eksik Değer Yüzdesi': missing_percentage
})
print(missing_df[missing_df['Eksik Değer Sayısı'] > 0])

## 4. Hedef Değişken Analizi (Şikayet Kategorileri)

In [ ]:
# Kategori dağılımını analiz et
category_counts = df['complaint_category'].value_counts()
category_percentages = (category_counts / len(df)) * 100

print("=== Şikayet Kategorileri Dağılımı ===")
for category, count in category_counts.items():
    percentage = category_percentages[category]
    print(f"{category}: {count:,} ({percentage:.1f}%)")

# Görselleştirme
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Bar plot
category_counts.plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Şikayet Kategorileri Dağılımı', fontsize=14, fontweight='bold')
ax1.set_xlabel('Kategori')
ax1.set_ylabel('Sayı')
ax1.tick_params(axis='x', rotation=45)

# Pie chart
colors = plt.cm.Set3(np.linspace(0, 1, len(category_counts)))
ax2.pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%', 
        colors=colors, startangle=90)
ax2.set_title('Şikayet Kategorileri Yüzde Dağılımı', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 5. Metin Verisi Analizi

In [ ]:
# Metin uzunluğu analizi
df['text_length'] = df['complaint_text'].str.len()
df['word_count'] = df['complaint_text'].str.split().str.len()

print("=== Metin Uzunluğu İstatistikleri ===")
print(f"Ortalama metin uzunluğu: {df['text_length'].mean():.1f} karakter")
print(f"Medyan metin uzunluğu: {df['text_length'].median():.1f} karakter")
print(f"Minimum metin uzunluğu: {df['text_length'].min()} karakter")
print(f"Maksimum metin uzunluğu: {df['text_length'].max()} karakter")

print("\n=== Kelime Sayısı İstatistikleri ===")
print(f"Ortalama kelime sayısı: {df['word_count'].mean():.1f} kelime")
print(f"Medyan kelime sayısı: {df['word_count'].median():.1f} kelime")
print(f"Minimum kelime sayısı: {df['word_count'].min()} kelime")
print(f"Maksimum kelime sayısı: {df['word_count'].max()} kelime")

In [ ]:
# Metin uzunluğu dağılımını görselleştir
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Karakter uzunluğu histogramı
axes[0, 0].hist(df['text_length'], bins=50, color='lightblue', alpha=0.7)
axes[0, 0].set_title('Metin Uzunluğu Dağılımı (Karakter)', fontweight='bold')
axes[0, 0].set_xlabel('Karakter Sayısı')
axes[0, 0].set_ylabel('Frekans')

# Kelime sayısı histogramı
axes[0, 1].hist(df['word_count'], bins=50, color='lightgreen', alpha=0.7)
axes[0, 1].set_title('Kelime Sayısı Dağılımı', fontweight='bold')
axes[0, 1].set_xlabel('Kelime Sayısı')
axes[0, 1].set_ylabel('Frekans')

# Kategoriye göre metin uzunluğu box plot
df.boxplot(column='text_length', by='complaint_category', ax=axes[1, 0])
axes[1, 0].set_title('Kategoriye Göre Metin Uzunluğu')
axes[1, 0].set_xlabel('Kategori')
axes[1, 0].set_ylabel('Karakter Sayısı')
axes[1, 0].tick_params(axis='x', rotation=45)

# Kategoriye göre kelime sayısı box plot
df.boxplot(column='word_count', by='complaint_category', ax=axes[1, 1])
axes[1, 1].set_title('Kategoriye Göre Kelime Sayısı')
axes[1, 1].set_xlabel('Kategori')
axes[1, 1].set_ylabel('Kelime Sayısı')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Kategoriye Göre Metin Uzunluğu İstatistikleri

In [ ]:
# Kategoriye göre metin uzunluğu istatistikleri
text_stats_by_category = df.groupby('complaint_category').agg({
    'text_length': ['mean', 'median', 'std', 'min', 'max'],
    'word_count': ['mean', 'median', 'std', 'min', 'max']
}).round(2)

print("=== Kategoriye Göre Metin Uzunluğu İstatistikleri ===")
print(text_stats_by_category)

## 7. Diğer Özelliklerin Analizi

In [ ]:
# Ürün tipi dağılımı
print("=== Ürün Tipi Dağılımı ===")
product_counts = df['product_type'].value_counts()
print(product_counts)

# Şikayet kanalı dağılımı
print("\n=== Şikayet Kanalı Dağılımı ===")
channel_counts = df['complaint_channel'].value_counts()
print(channel_counts)

# Öncelik seviyesi dağılımı
print("\n=== Öncelik Seviyesi Dağılımı ===")
priority_counts = df['priority_level'].value_counts()
print(priority_counts)

In [ ]:
# Diğer özelliklerin görselleştirilmesi
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Ürün tipi dağılımı
product_counts.head(10).plot(kind='bar', ax=axes[0, 0], color='coral')
axes[0, 0].set_title('En Popüler 10 Ürün Tipi', fontweight='bold')
axes[0, 0].set_xlabel('Ürün Tipi')
axes[0, 0].set_ylabel('Sayı')
axes[0, 0].tick_params(axis='x', rotation=45)

# Şikayet kanalı dağılımı
channel_counts.plot(kind='bar', ax=axes[0, 1], color='lightgreen')
axes[0, 1].set_title('Şikayet Kanalı Dağılımı', fontweight='bold')
axes[0, 1].set_xlabel('Kanal')
axes[0, 1].set_ylabel('Sayı')
axes[0, 1].tick_params(axis='x', rotation=45)

# Öncelik seviyesi dağılımı
priority_counts.plot(kind='bar', ax=axes[1, 0], color='gold')
axes[1, 0].set_title('Öncelik Seviyesi Dağılımı', fontweight='bold')
axes[1, 0].set_xlabel('Öncelik')
axes[1, 0].set_ylabel('Sayı')
axes[1, 0].tick_params(axis='x', rotation=45)

# Müşteri memnuniyeti dağılımı
satisfaction_counts = df['satisfaction_rating'].value_counts().sort_index()
satisfaction_counts.plot(kind='bar', ax=axes[1, 1], color='plum')
axes[1, 1].set_title('Müşteri Memnuniyeti Dağılımı', fontweight='bold')
axes[1, 1].set_xlabel('Memnuniyet Puanı (1-5)')
axes[1, 1].set_ylabel('Sayı')

plt.tight_layout()
plt.show()

## 8. Kategori ve Diğer Özellikler Arasındaki İlişkiler

In [ ]:
# Kategori ile öncelik seviyesi arasındaki ilişki
category_priority_crosstab = pd.crosstab(df['complaint_category'], df['priority_level'])
print("=== Kategori vs Öncelik Seviyesi ===")
print(category_priority_crosstab)

# Kategori ile memnuniyet puanı arasındaki ilişki
category_satisfaction = df.groupby('complaint_category')['satisfaction_rating'].agg(['mean', 'std']).round(2)
print("\n=== Kategoriye Göre Ortalama Memnuniyet Puanı ===")
print(category_satisfaction)

In [ ]:
# Kategori-öncelik ilişkisini görselleştir
plt.figure(figsize=(14, 8))
category_priority_pct = pd.crosstab(df['complaint_category'], df['priority_level'], normalize='index') * 100
category_priority_pct.plot(kind='bar', stacked=True, ax=plt.gca())
plt.title('Kategoriye Göre Öncelik Seviyesi Dağılımı (%)', fontsize=14, fontweight='bold')
plt.xlabel('Kategori')
plt.ylabel('Yüzde (%)')
plt.legend(title='Öncelik Seviyesi')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Kategoriye göre memnuniyet puanını görselleştir
plt.figure(figsize=(12, 6))
category_satisfaction['mean'].plot(kind='bar', color='lightcoral')
plt.title('Kategoriye Göre Ortalama Müşteri Memnuniyeti', fontsize=14, fontweight='bold')
plt.xlabel('Kategori')
plt.ylabel('Ortalama Memnuniyet Puanı')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 9. Zaman Analizi

In [ ]:
# Tarih sütununu datetime'a çevir
df['complaint_date'] = pd.to_datetime(df['complaint_date'])
df['year'] = df['complaint_date'].dt.year
df['month'] = df['complaint_date'].dt.month
df['day_of_week'] = df['complaint_date'].dt.day_name()

# Aylık şikayet trendi
monthly_complaints = df.groupby(['year', 'month']).size().reset_index(name='complaint_count')
monthly_complaints['date'] = pd.to_datetime(monthly_complaints[['year', 'month']].assign(day=1))

plt.figure(figsize=(14, 6))
plt.plot(monthly_complaints['date'], monthly_complaints['complaint_count'], marker='o', linewidth=2)
plt.title('Aylık Şikayet Trendi', fontsize=14, fontweight='bold')
plt.xlabel('Tarih')
plt.ylabel('Şikayet Sayısı')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Haftanın günlerine göre dağılım
print("=== Haftanın Günlerine Göre Şikayet Dağılımı ===")
day_counts = df['day_of_week'].value_counts()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_counts = day_counts.reindex(day_order)
print(day_counts)

## 10. En Sık Kullanılan Kelimeler ve N-gram Analizi

In [ ]:
from collections import Counter
import re

# Türkçe stop words (basit versiyon)
turkish_stopwords = {
    've', 'ile', 'için', 'de', 'da', 'bu', 'o', 'bir', 'çok', 'var', 'yok', 'ama', 'ki', 'mi', 'mu',
    'sen', 'seni', 'sana', 'biz', 'bizi', 'bize', 'onlar', 'onları', 'onlara', 'o', 'onu', 'ona',
    'ne', 'nerede', 'nasıl', 'kim', 'hangi', 'kadar', 'kime', 'kimin', 'neden', 'niçin', 'ne zaman'
}

def extract_words(text):
    """Metinden kelimeleri çıkar ve stop words'leri filtrele"""
    # Küçük harfe çevir ve özel karakterleri temizle
    text = re.sub(r'[^\w\s]', ' ', text.lower())
    words = text.split()
    # Stop words ve 2 karakterden kısa kelimeleri filtrele
    words = [word for word in words if word not in turkish_stopwords and len(word) > 2]
    return words

# Tüm metinlerden kelimeleri çıkar
all_words = []
for text in df['complaint_text']:
    all_words.extend(extract_words(text))

# En sık 20 kelimeyi bul
word_counts = Counter(all_words)
top_words = word_counts.most_common(20)

print("=== En Sık Kullanılan 20 Kelime ===")
for word, count in top_words:
    print(f"{word}: {count:,}")

# Görselleştir
plt.figure(figsize=(12, 8))
words, counts = zip(*top_words)
plt.barh(words, counts, color='skyblue')
plt.title('En Sık Kullanılan 20 Kelime', fontsize=14, fontweight='bold')
plt.xlabel('Frekans')
plt.ylabel('Kelime')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 11. Kategori Bazında En Sık Kelimeler

In [ ]:
# Her kategori için en sık kelimeleri bul
fig, axes = plt.subplots(3, 3, figsize=(20, 15))
axes = axes.flatten()

categories = df['complaint_category'].unique()

for i, category in enumerate(categories):
    category_texts = df[df['complaint_category'] == category]['complaint_text']
    category_words = []
    
    for text in category_texts:
        category_words.extend(extract_words(text))
    
    category_word_counts = Counter(category_words)
    top_category_words = category_word_counts.most_common(10)
    
    if top_category_words:
        words, counts = zip(*top_category_words)
        axes[i].barh(words, counts, color=plt.cm.Set3(i))
        axes[i].set_title(f'{category}', fontweight='bold')
        axes[i].set_xlabel('Frekans')
        axes[i].invert_yaxis()

plt.tight_layout()
plt.show()

## 12. EDA Bulguları ve Özet

### Veri Seti Özeti:
- **Toplam Kayıt**: 12,000 müşteri şikayeti
- **Özellik Sayısı**: 11 adet (metin verisi dahil)
- **Hedef Değişken**: 9 farklı şikayet kategorisi
- **Eksik Değer**: Yok

### Temel Bulgular:

#### 1. Kategori Dağılımı:
- **En yaygın kategori**: Delivery Issues (%25)
- **İkinci sırada**: Billing Issues (%20)
- **En az görülen**: Fraud Issues (%2)

#### 2. Metin Özellikleri:
- **Ortalama metin uzunluğu**: ~80 karakter
- **Ortalama kelime sayısı**: ~12 kelime
- **Metin uzunluğu kategorilere göre değişkenlik gösteriyor

#### 3. Müşteri Demografisi:
- **Yaş aralığı**: 18-70
- **Müşteri deneyimi**: 1-60 ay
- **Memnuniyet puanı**: 1-5 arası (1=çok memnun değil, 5=çok memnun)

#### 4. İş Kanal Analizi:
- **En çok kullanılan kanal**: Phone (%~30)
- **İkinci sırada**: Email (%~25)
- **En az kullanılan**: Mobile App (%~10)

#### 5. Öncelik Dağılımı:
- **Critical ve High** öncelikli şikayetler daha az ama kritik
- **Low ve Medium** öncelikli şikayetler çoğunlukta

### Modelleme İçin Çıkarımlar:

1. **Class Imbalance**: Delivery Issues ve Billing Issues kategorileri diğerlerinden çok daha fazla. Bu durum için stratifikasyon veya class weighting gerekebilir.

2. **Metin Uzunluğu**: Şikayet metinleri ortalama 12 kelime ile kısa-orta uzunlukta. Bu, TF-IDF ve basit NLP tekniklerinin etkili olabileceğini gösteriyor.

3. **Kategori Belirginliği**: Her kategorinin kendine özgü kelime dağılımı var. Bu, NLP modellerinin kategorileri ayırt edebileceğini gösteriyor.

4. **Çoklu Özellik**: Metin verisi yanında sayısal ve kategorik özellikler de mevcut. Bu, hibrit modelleme yaklaşımlarını destekliyor.

### Önerilen Modelleme Stratejileri:

1. **Basit Modeller**: TF-IDF + Logistic Regression, Naive Bayes
2. **Gelişmiş Modeller**: Word2Vec + SVM, BERT-based models
3. **Ensemble Yöntemler**: Voting Classifier, Random Forest
4. **Çoklu Modal**: Metin + sayısal özellikler birleştirmesi

### Sonraki Adımlar:

1. Baseline model geliştirme
2. Feature engineering (metin temizleme, n-gram, sentiment analysis)
3. Model optimizasyonu ve hiperparametre tuning
4. Cross-validation ve model değerlendirme
5. Final pipeline oluşturma